<a href="https://colab.research.google.com/github/emmahoggett/Error_class_lenstronomy/blob/master/NeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/deeplens/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g4FxL5QRsmrGZzI2Qn3ECaMQ-cKsdwORPuUtqTqXc59D8WGNwFgoHk


In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install lenstronomy
!pip3 install deeplenstronomy
!pip install h5py
!pip install matplotlib==3.1.3
!pip install --upgrade tables

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 25kB/s 
     |████████████████████████████████| 8.8MB 33.6MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.2.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
     |████████████████████████████████| 440kB 4.1MB/s 
  Created wheel for lenstronomy: filename=lenstronomy-1.7.0-cp37-none-any.whl size=632852 sha256=3fdcce30d272a987d1a9d11109aade0e7d99930203c353729ca92dd61d12016a
  Stored in directory: /root/.cache/pip/wheels/80/36/eb/a1bbf501dc372452b939df565ca4da0b5d0f1001b05739ae46
Successfully built lenstronomy
     |████████████████████████████████| 61kB 2.7MB/s 
     |█████████

## 0. Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import PIL

import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from helpers import*
from model.baseline import*
from model.densenet121 import*
from model.alexnet import*
from model.resnet18 import*
from model.vgg16 import*

## 1. Building the data set

In [9]:
# Build the four classes 
config_repo_model = 'drive/My Drive/Colab Notebooks/deeplens/data/configFile/config_model'


for i in np.arange(1,4):
    model_name = config_repo_model + str(i) + '.yaml'
    res = Residual(model_name)
    res.build(i)

print('Data Generation Finished')

Data Generation Finished


In [78]:
metadata = pd.DataFrame()
for i in np.arange(1,4):
    [img, meta] = read_hdf5(i)
    metadata = pd.concat([metadata,meta], ignore_index=True)
    if i == 1:
        final_array = img
    else:
         final_array = np.concatenate((final_array, img))

data_set = CombineDataset(metadata,'ID','class',final_array)

In [80]:
class CombineDataset(Dataset):
    """
    This class helps us to build a pytorch tensor by combining the images and the
    metadata.
    """

    def __init__(self, frame, id_col, label_name, images, nb_channel = 1):
        """
        Args:
            frame (pd.DataFrame): Frame with the tabular data.
            id_col (string): Name of the column that connects image to tabular data
            label_name (string): Name of the column with the label to be predicted
            path_imgs (string): Path to the folder where the images are.
            nb_channel (int): Number of channels.
        """
        self.frame = frame
        self.id_col = id_col
        self.label_name = label_name
        self.images = images
        self.nb_channel = nb_channel
        
    def __len__(self):
        return (self.frame.shape[0])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #complete image path and read
        img_name = self.frame[self.id_col].iloc[idx]
        image = image[img_name]
        image = torch.from_numpy(image.astype(np.float64))

        #get the other features to be used as training data
        feats = [feat for feat in self.frame.columns if feat not in [self.label_name,self.id_col]]
        feats  = np.array(self.frame[feats].iloc[idx])
        feats = torch.from_numpy(feats.astype(np.float64))
       
        
        #get label
        label = np.array(self.frame[self.label_name].iloc[idx])
        label = torch.from_numpy(label.astype(np.float64))

        return image, feats, label
        
    def size(self):
        return (self.frame.shape[0])

AttributeError: ignored

In [76]:
data_train, data_test = train_test_split(data_set,train_size=0.9,
                                         random_state=42)

KeyError: ignored

In [72]:
img_name = metadata['ID'].iloc[1]
print(img_name)

1


In [35]:
# Open and concatenate the metadata in one data frame
data_repo = '/content/drive/My Drive/Colab Notebooks/deeplens/data/dataSet/'
metadata = pd.DataFrame()
for i in np.arange(1,4):
    file_name = data_repo+'MetaE'+str(i)+'.csv'
    meta = pd.read_csv(file_name)
    metadata = pd.concat([metadata,meta], ignore_index=True)



# Combine the residuals with the metadata in a pytorch tensor
data_set = CombineDataset(metadata,'ID','class',data_repo)

data_train, data_test = train_test_split(data_set,train_size=0.9,
                                         random_state=42)

loader_train = DataLoader(data_train, batch_size = 4, shuffle = True, 
                          num_workers = 0, drop_last=True)
loader_test = DataLoader(data_test, batch_size = 4, shuffle = True, 
                         num_workers = 0, drop_last=True)

ValueError: ignored

In [52]:
data_set

<bound method CombineDataset.size of <helpers.CombineDataset object at 0x7f9bec9c2b10>>

In [ ]:
loader_train = DataLoader(data_train, batch_size = 4, shuffle = True, 
                          num_workers = 0, drop_last=True)
loader_test = DataLoader(data_test, batch_size = 4, shuffle = True, 
                         num_workers = 0, drop_last=True)

## 2. Building basic neural network

#### 2.1. Tabular network

In [ ]:
net = TabularNetBasic()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(40):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(meta_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        outputs = net(meta_img)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

iter: 30 
 test:
 accuracy: 0.638 samples f1: 0.650


#### 2.2. Residual image 

In [ ]:
net = CNNNetBasic()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(40):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        outputs = net(images)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

iter: 30 
 test:
 accuracy: 0.590 samples f1: 0.593


#### 2.3. Residual and tabular data

In [ ]:
net = TabularCNNNetBasic()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(35):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs,meta_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        outputs = net(images,meta_img)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

iter: 30 
 test:
 accuracy: 0.744 samples f1: 0.749


## 3. Building different model

#### 3.1. AlexNet

In [ ]:
squeezenet = models.squeezenet1_0()
inception = models.inception_v3()
shufflenet = models.shufflenet_v2_x1_0()
mobilenet = models.mobilenet_v2()
resnext50_32x4d = models.resnext50_32x4d()
wide_resnet50_2 = models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()

###### 3.1.1. Residual maps

In [ ]:
net = AlexNetResidual()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

###### 3.1.2. Residual maps & Metadata


In [ ]:
net = AlexNetResidualMetadata()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs, meta_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images,meta_img)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

iter: 29 
 test:
 accuracy: 0.467 samples f1: 0.474


#### 3.2. Resnet18

###### 3.2.1. Residual maps

In [ ]:
net = resnet18maps(1,3)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

iter: 29 
 test:
 accuracy: 0.105 samples f1: 0.119


###### 3.2.2. Residual maps & Metadata

In [ ]:
net = resnet18meta(1,7,3)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs, meta_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images,meta_img)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

iter: 29 
 test:
 accuracy: 0.652 samples f1: 0.667


#### 3.3. VGG16

 ###### 3.3.1. Residual maps

In [ ]:
net = VGG16Residual()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

###### 3.3.2. Residual maps & Metadata

In [ ]:
net = VGG16ResidualMetadata()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs,meta_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images,meta_img)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

#### 3.4. Dense Net 121

###### 3.4.1. Residual maps

In [ ]:
net = densenet121()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = inputs.resize_([4,1,224,224])
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        images = images.resize_([4,1,224,224])
        outputs = net(images)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("iter:{:3d} \n test:\n "
        "accuracy: {:.3f} "
        "samples f1: {:.3f}".format(iteration,
                                    mean_accuracy,
                                    mean_f1))

###### 3.4.2. Residual maps & Metadata

#### 3.5. Google Net

###### 3.5.1. Residual maps

###### 3.5.2. Residual maps & Metadata

#### 3.6. Squeeze Net 1_0

###### 3.6.1. Residual maps

###### 3.6.2. Residual maps & Metadata